In [15]:
import re
from urllib.parse import urlparse
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from datetime import datetime
import time
from openpyxl import load_workbook

# try to import known scrapers
try:
    from chrisdetzel_hbm import scrape_hbm_product
except Exception:
    scrape_hbm_product = None
    
try:
    from chrisdetzel_holzprofi import scrape_holzprofi_product
except Exception:
    scrape_holzprofi_product = None
        
try:
    from chrisdetzel_neureiter import scrape_neureiter_product
except Exception:
    scrape_neureiter_product = None

try:
    from chrisdetzel_idealo import scrape_idealo_product
except Exception:
    scrape_idealo_product = None

def setup_headless_driver():
    options = Options()
    #options.add_argument("--headless")
    options.add_argument("--window-size=1920,1080")
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()), options=options
    )
    return driver


def _choose_url_column(df):
    # Prefer column F (index 5). If not present, try to find a column containing 'url'
    if len(df.columns) >= 6:
        return df.columns[5]
    for c in df.columns:
        if 'url' in str(c).lower():
            return c
    raise ValueError('Konnte keine URL-Spalte finden (erwarte Spalte F oder eine Spalte mit "url").')

def _domain_from_url(url):
    try:
        netloc = urlparse(str(url)).netloc.lower()
        if netloc.startswith('www.'):
            netloc = netloc[4:]
        return netloc
    except Exception:
        return ''

def _make_new_column_name(old_name, today_str):
    return f"{old_name}_{today_str}"



In [16]:
path = '/Users/gabrielhipp/Library/Mobile Documents/com~apple~CloudDocs/fiverr/chrisdetzel/Preisanalyse Wettbewerber v2.1 copy.xlsx'

In [17]:
# Read sheet
sheet = 'Option 1 - nebeneinander'
# Die erste Zeile ist leer und die zweite Zeile enthält die Header.
# Daher header=1 verwenden
df = pd.read_excel(path, sheet_name=sheet, engine='openpyxl', header=1)

url_col = _choose_url_column(df)

# Determine last 4 columns to copy names from
last4 = ["Status", "Preis", "Lieferstatus", "Versandkosten"]

today_str = datetime.today().strftime('%Y-%m-%d_%H-%M-%S')
new_col_names = [_make_new_column_name(str(c), today_str) for c in last4]

# Prepare empty series for new columns
new_cols = {name: pd.Series([pd.NA] * len(df), index=df.index, dtype=object) for name in new_col_names}

# Domain -> function dispatch
dispatch = {
    'hbm-machines.com': scrape_hbm_product,
    'holzprofi.com': scrape_holzprofi_product,
    'neureiter-shop.at': scrape_neureiter_product,
    'idealo.de': scrape_idealo_product,
    'idealo.at': scrape_idealo_product,
}


In [18]:

for idx, row in df.iterrows():
    url = row.get(url_col)
    if not isinstance(url, str) or not url.strip():
        continue

    domain = _domain_from_url(url)

    scraper = None
    # try exact match and suffix match
    if domain in dispatch and dispatch[domain]:
        scraper = dispatch[domain]
        print("Using exact match for domain:", domain)
    else:
        for k, v in dispatch.items():
            if k and k in domain and v:
                scraper = v
                break

    scraped = None
    if scraper is None:
        print(f"No scraper available for this domain {domain} -> skip")
        continue

    try:
        driver = setup_headless_driver()
        driver.get(url)
        scraped = scraper(driver)
        driver.quit()
        print(f"Scraped data for URL {url}: {scraped}")
    except Exception:
        scraped = None

    if not scraped:
        continue

    # Map scraped results into new columns heuristically
    # Determine a simple mapping based on keywords in last4 column names
    for new_col in new_col_names:
        base = str(new_col).lower()
        val = None
        if 'preis' in base:
            val = scraped['price']
        elif 'lieferstatus' in base:
            val = scraped['availability']
        elif 'versandkosten' in base:
            val = scraped['shipping_costs']
        else: "-"
        print(f"val={val}")
        new_cols[new_col].iat[idx] = val



Using exact match for domain: hbm-machines.com
Preis: 349,99
Stock: 
Scraped data for URL https://www.hbm-machines.com/at/p/hbm-200-entstaubungsanlage-230-volt: {'price': '349,99', 'availability': '', 'shipping_costs': '-'}
val=None
val=349,99
val=
val=-
Using exact match for domain: idealo.de
Successfully clicked deny button in shadow DOM
Price number: 208
Scraped data for URL https://www.idealo.de/preisvergleich/OffersOfProduct/6428107_-absauganlage-abs-1080-400-v-holzmann.html: {'price': 208, 'availability': '-', 'shipping_costs': '-'}
val=None
val=208
val=-
val=-
Using exact match for domain: holzprofi.com
Preis: 259,00
Stock: AUF LAGER
Scraped data for URL https://holzprofi.com/products/absaugung-hans-schreiner-fm230a-hohere-ausf/: {'price': '259,00', 'availability': 'AUF LAGER', 'shipping_costs': '-'}
val=None
val=259,00
val=AUF LAGER
val=-
Using exact match for domain: neureiter-shop.at
Price: 390
Stock: Lagernd
Scraped data for URL https://neureiter-shop.at/bt400-absauggerat/: 

In [19]:
test_output_path = path = '/Users/gabrielhipp/Library/Mobile Documents/com~apple~CloudDocs/fiverr/chrisdetzel/Preisanalyse Wettbewerber v2.1 copy.xlsx'

# Append new columns to df (they will be added at the end)
for name, series in new_cols.items():
    # if name collides, generate unique
    final_name = name
    counter = 1
    while final_name in df.columns:
        final_name = f"{name}_{counter}"
        counter += 1
    df[final_name] = series

# Write back replacing only the sheet, preserving the rest of the workbook
try:
    # pandas >= 1.3 supports if_sheet_exists='replace'
    with pd.ExcelWriter(path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        df.to_excel(writer, sheet_name=sheet, index=False)
except TypeError:
    # older pandas: load workbook, remove sheet then write
    
    wb = load_workbook(path)
    if sheet in wb.sheetnames:
        std = wb[sheet]
        wb.remove(std)
    wb.save(path)
    with pd.ExcelWriter(path, engine='openpyxl', mode='a') as writer:
        df.to_excel(writer, sheet_name=sheet, index=False)